In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
import copy
import math

In [3]:
df = pd.read_csv("G:\\Best_risk_reward\\Dec2020\\Date-24-12-2020\\Data\\Adjusted_Closed_Price_For_all_Dec_24.csv",)

In [5]:
import copy
data_df = copy.deepcopy(df)
df = data_df

In [6]:
df.dtypes

Symbol          object
Date            object
Close Price    float64
year             int64
month            int64
dtype: object

In [7]:
df['Date'] = pd.to_datetime(df['Date'],errors='coerce',exact=True,format='%Y-%m-%d') 

In [8]:
# leat's crate one feature year
df["yr"] =  pd.DatetimeIndex(df['Date']).year

In [9]:
df.dtypes

Symbol                 object
Date           datetime64[ns]
Close Price           float64
year                    int64
month                   int64
yr                      int64
dtype: object

In [10]:
df.head()

,Symbol,Date,Close Price,year,month,yr
0,ADANIPORTS,2007-11-27,962.90,2007,11,2007
1,ADANIPORTS,2007-11-28,893.90,2007,11,2007
2,ADANIPORTS,2007-11-29,884.20,2007,11,2007
3,ADANIPORTS,2007-11-30,921.55,2007,11,2007
4,ADANIPORTS,2007-12-03,969.30,2007,12,2007


# calculate return
     df['Return']= abs(df['Adj_Close_Price']/df['Adj_Close_Price'].shift()-1)
     df['Return1']= df['Adj_Close_Price']/df['Adj_Close_Price'].shift()-1

In [11]:
df.head()

,Symbol,Date,Close Price,year,month,yr
0,ADANIPORTS,2007-11-27,962.90,2007,11,2007
1,ADANIPORTS,2007-11-28,893.90,2007,11,2007
2,ADANIPORTS,2007-11-29,884.20,2007,11,2007
3,ADANIPORTS,2007-11-30,921.55,2007,11,2007
4,ADANIPORTS,2007-12-03,969.30,2007,12,2007


In [12]:
df.rename(columns={'Close Price':"Adj_Close_Price"},inplace=True)

In [13]:
data_df = pd.DataFrame()
for com in list(df.Symbol.unique()):
    df1=df.loc[df.Symbol.isin([com])]
    df1.loc[:,'Return']= abs(df1.loc[:,'Adj_Close_Price'].pct_change())
    df1.loc[:,'Return1']= df1.loc[:,'Adj_Close_Price'].pct_change()
    df1.dropna(axis=0,inplace=True)
    data_df = data_df.append(df1)

C:\python\Anaconda3\envs\intelPython\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\python\Anaconda3\envs\intelPython\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\python\Anaconda3\envs\intelPython\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the

In [14]:
data_df.head()

,Symbol,Date,Adj_Close_Price,year,month,yr,Return,Return1
1,ADANIPORTS,2007-11-28,893.90,2007,11,2007,0.071659,-0.071659
2,ADANIPORTS,2007-11-29,884.20,2007,11,2007,0.010851,-0.010851
3,ADANIPORTS,2007-11-30,921.55,2007,11,2007,0.042242,0.042242
4,ADANIPORTS,2007-12-03,969.30,2007,12,2007,0.051815,0.051815
5,ADANIPORTS,2007-12-04,1041.45,2007,12,2007,0.074435,0.074435


In [15]:
# lets check duplicate in the data
print("Toatl duplicate data in the data set:",data_df.duplicated(subset=None, keep='first').sum())
duplicateRowsDF = data_df[data_df.duplicated(subset=None, keep='first')]
print("Name of duplicate row:\n",duplicateRowsDF)
# drop duplicate row
data_df.drop_duplicates(keep='first',inplace=True)
print('Data Shape before drop na values :', data_df.shape)
data_df.dropna(axis=0,inplace=True)
print('Data shape after drop na values',data_df.shape)

Toatl duplicate data in the data set: 0
Name of duplicate row:
 Empty DataFrame
Columns: [Symbol, Date, Adj_Close_Price, year, month, yr, Return, Return1]
Index: []
Data Shape before drop na values : (150429, 8)
Data shape after drop na values (150429, 8)


In [16]:
# copy the data
np.random.seed(5)
data = copy.copy(data_df)

In [17]:
data.head()

,Symbol,Date,Adj_Close_Price,year,month,yr,Return,Return1
1,ADANIPORTS,2007-11-28,893.90,2007,11,2007,0.071659,-0.071659
2,ADANIPORTS,2007-11-29,884.20,2007,11,2007,0.010851,-0.010851
3,ADANIPORTS,2007-11-30,921.55,2007,11,2007,0.042242,0.042242
4,ADANIPORTS,2007-12-03,969.30,2007,12,2007,0.051815,0.051815
5,ADANIPORTS,2007-12-04,1041.45,2007,12,2007,0.074435,0.074435


In [18]:
data.shape

(150429, 8)

# Drop 1st row of the each company 

    def mask_first(x):
        result = np.ones_like(x)
        result[0] = 0
        return result

    mask = data.groupby('Symbol')['Symbol'].transform(mask_first).astype(bool)
    data=data.loc[mask]
    print('After drop data shape', data.shape)

data.head()

# Create Data Dict 
     Data Dictionary of the each comapny

In [19]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
Company_name=list(data.Symbol.unique())
import xlsxwriter

In [20]:
data_disc = []
data_name = []
for row in Company_name:
    data_as = data[data['Symbol']==row]
    print('='*50+'Data information of ' + str(row)+'='*50)
    print(data_as.info(),'\n')
    print('='*50+"Data discription of "+ str(row)+'='*50 )
    print(str(row)+ ':\n' ,data_as.drop('yr',axis=1).describe())
    data_disc.append(data_as.drop('yr',axis=1).describe()) 
    data_name.append(row)

==================================================Data information of ADANIPORTS==================================================
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2824 entries, 1 to 2824
Data columns (total 8 columns):
Symbol             2824 non-null object
Date               2824 non-null datetime64[ns]
Adj_Close_Price    2824 non-null float64
year               2824 non-null int64
month              2824 non-null int64
yr                 2824 non-null int64
Return             2824 non-null float64
Return1            2824 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 198.6+ KB
None 

==================================================Data discription of ADANIPORTS==================================================
ADANIPORTS:
        Adj_Close_Price         year        month       Return      Return1
count      2824.000000  2824.000000  2824.000000  2824.000000  2824.000000
mean       1092.627780  2013.123229     6.428470   

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5062 entries, 27286 to 32347
Data columns (total 8 columns):
Symbol             5062 non-null object
Date               5062 non-null datetime64[ns]
Adj_Close_Price    5062 non-null float64
year               5062 non-null int64
month              5062 non-null int64
yr                 5062 non-null int64
Return             5062 non-null float64
Return1            5062 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 355.9+ KB
None 

==================================================Data discription of CIPLA==================================================
CIPLA:
        Adj_Close_Price         year        month       Return      Return1
count      5062.000000  5062.000000  5062.000000  5062.000000  5062.000000
mean      11485.072723  2008.628803     6.450415     0.014152     0.000852
std        7792.123602     5.871130     3.450988     0.014896     0.020530
min         900.000000  1999.000000  

        Adj_Close_Price         year        month       Return      Return1
count      5062.000000  5062.000000  5062.000000  5062.000000  5062.000000
mean       8674.758979  2008.628803     6.450415     0.013508     0.000876
std        6966.303324     5.871130     3.450988     0.014066     0.019483
min         550.000000  1999.000000     1.000000     0.000000    -0.120486
25%        2579.312500  2004.000000     3.000000     0.004008    -0.008926
50%        5264.625000  2009.000000     6.000000     0.009496     0.000087
75%       13151.312500  2014.000000     9.000000     0.018078     0.009952
max       24184.750000  2019.000000    12.000000     0.191224     0.191224
==================================================Data information of HINDALCO==================================================
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5062 entries, 62467 to 67528
Data columns (total 8 columns):
Symbol             5062 non-null object
Date               5062 non-null datetime64[

max         293.050000  2019.000000    12.000000     0.134895     0.123047
==================================================Data information of ONGC==================================================
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5062 entries, 99612 to 104673
Data columns (total 8 columns):
Symbol             5062 non-null object
Date               5062 non-null datetime64[ns]
Adj_Close_Price    5062 non-null float64
year               5062 non-null int64
month              5062 non-null int64
yr                 5062 non-null int64
Return             5062 non-null float64
Return1            5062 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 355.9+ KB
None 

==================================================Data discription of ONGC==================================================
ONGC:
        Adj_Close_Price         year        month       Return      Return1
count      5062.000000  5062.000000  5062.000000  5062.000000  

==================================================Data information of UPL==================================================
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3789 entries, 128065 to 131853
Data columns (total 8 columns):
Symbol             3789 non-null object
Date               3789 non-null datetime64[ns]
Adj_Close_Price    3789 non-null float64
year               3789 non-null int64
month              3789 non-null int64
yr                 3789 non-null int64
Return             3789 non-null float64
Return1            3789 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 266.4+ KB
None 

==================================================Data discription of UPL==================================================
UPL:
        Adj_Close_Price         year        month       Return      Return1
count      3789.000000  3789.000000  3789.000000  3789.000000  3789.000000
mean       2985.834086  2011.177619     6.453946     0.017776    

In [21]:
writer = pd.ExcelWriter('G:\\Best_risk_reward\\Dec2020\\Date-24-12-2020\\Data\\discriptive_data_Dec24_2020.xlsx', engine='xlsxwriter')
for name,df in zip(data_name,data_disc):
    df.to_excel(writer, sheet_name='Data dicription '+name)
writer.save()

In [22]:
B_data = copy.copy(data)

In [23]:
#total number of bins
b= [-0.9,0,0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,1,2,3,4,5,6,7,8,9,10]

In [24]:
len(b)

31

In [25]:
l = [i for i in range(1,len(b))] # label
B_data['binned']=pd.cut(x=B_data['Return'], bins=b,labels=l)
#B_data['binned']=pd.cut(x=B_data['Return'], bins=b,labels=l)

In [26]:
B_data.binned.unique()

[4, 3, 2, 6, 5, 7, 1, 8, 15]
Categories (9, int64): [1 < 2 < 3 < 4 ... 6 < 7 < 8 < 15]

In [27]:
B_data.to_csv('G:\\Best_risk_reward\\Dec2020\\Date-24-12-2020\\Data\\df_b_data_Dec24_2020.csv',encoding='utf-8')

In [28]:
B_data.head()

,Symbol,Date,Adj_Close_Price,year,month,yr,Return,Return1,binned
1,ADANIPORTS,2007-11-28,893.90,2007,11,2007,0.071659,-0.071659,4
2,ADANIPORTS,2007-11-29,884.20,2007,11,2007,0.010851,-0.010851,3
3,ADANIPORTS,2007-11-30,921.55,2007,11,2007,0.042242,0.042242,3
4,ADANIPORTS,2007-12-03,969.30,2007,12,2007,0.051815,0.051815,4
5,ADANIPORTS,2007-12-04,1041.45,2007,12,2007,0.074435,0.074435,4


In [29]:
# create the bins for return
pivot_n= B_data.pivot_table(index=['Symbol'],values=["Return"],columns=['binned'],aggfunc=lambda x: len(x),
                            margins_name='Count',margins=True)
pivot_n.fillna(0,inplace=True)

In [30]:
# for filter

In [31]:
#B_data.loc[:,'Return'].max()

In [32]:
#idl = B_data[B_data.loc[:,'binned'] ==7].index.values

In [33]:
idl = B_data[B_data.loc[:,'Return']>0.3].index.values

In [34]:
#print(idl)

In [35]:
len(idl)

1

In [36]:
#i=20
#range(i-10,i+10)

In [37]:
arr1=[]
for i in idl:
    x = range(i-2,i+2)
    arr1.extend(x)

In [38]:
#len(idl)*20

In [39]:
len(arr1)

4

In [40]:
B_data.loc[arr1].head(60)

,Symbol,Date,Adj_Close_Price,year,month,yr,Return,Return1,binned
91584,KOTAKBANK,2000-03-16,151.667373,2000,3,2000,0.069300,-0.069300,4
91585,KOTAKBANK,2000-03-21,139.583673,2000,3,2000,0.079672,-0.079672,4
91586,KOTAKBANK,2001-07-02,54.263718,2001,7,2001,0.611246,-0.611246,15
91587,KOTAKBANK,2001-07-03,54.207252,2001,7,2001,0.001041,-0.001041,2


In [41]:
#B_data.filter(items=arr1,axis=0).to_csv('b.csv',index=False)

In [42]:
#B_data.loc[arr1].to_csv('B_data_Analysis.csv',index=True)

In [43]:
#B_data.loc[idl].tail(60)

In [44]:
#B_data.to_csv('B_data.csv',index=False)

In [45]:
# now pivot table information is correct

In [46]:
pivot_n

Return                                                           \
binned          1        2        3       4      5      6     7     8   15   
Symbol                                                                       
ADANIPORTS   16.0   1090.0   1540.0   162.0   11.0    3.0   2.0   0.0  0.0   
ASIANPAINT   28.0   2873.0   2069.0    89.0    3.0    0.0   0.0   0.0  0.0   
AXISBANK     63.0   1995.0   2658.0   319.0   17.0    7.0   2.0   1.0  0.0   
BAJFINANCE   90.0   2131.0   2479.0   258.0   28.0    8.0   0.0   0.0  0.0   
BHARTIARTL   25.0   1775.0   2273.0   192.0    8.0    2.0   0.0   1.0  0.0   
BRITANNIA    19.0   2882.0   2062.0    94.0    4.0    0.0   0.0   0.0  0.0   
CIPLA        22.0   2556.0   2318.0   158.0    8.0    0.0   0.0   0.0  0.0   
DRREDDY       9.0   2523.0   2322.0   194.0   12.0    2.0   0.0   0.0  0.0   
GRASIM        5.0   2443.0   2401.0   197.0   13.0    3.0   0.0   0.0  0.0   
HCLTECH       6.0   2017.0   2428.0   300.0   34.0   17.0   0.0   0.0  0.0   
HDFC         13.0   2347.0   2506.0   185.0    8.0    2.0   1.0   0.0  0.0   
HDFCBANK     36.0   2647.0   2188.0   181.0    7.0    1.0   2.0   0.0  0.0   
HEROMOTOCO   10.0   2622.0   2297.0   125.0    4.0    4.0   0.0   0.0  0.0   
HINDALCO     28.0   2027.0   2729.0   262.0   13.0    2.0   1.0   0.0  0.0   
HINDUNILVR   36.0   2754.0   2188.0    79.0    5.0    0.0   0.0   0.0  0.0   
ICICIBANK    33.0   2025.0   2644.0   329.0   25.0    5.0   1.0   0.0  0.0   
INFY          4.0   2438.0   2363.0   233.0   18.0    5.0   0.0   1.0  0.0   
ITC          22.0   2668.0   2241.0   126.0    5.0    0.0   0.0   0.0  0.0   
JSWSTEEL      8.0   1301.0   1956.0   203.0   24.0    3.0   0.0   1.0  0.0   
KOTAKBANK    41.0   1961.0   2334.0   356.0   29.0   16.0   3.0   0.0  1.0   
NTPC         44.0   2023.0   1470.0    51.0    2.0    0.0   0.0   0.0  0.0   
ONGC         22.0   2594.0   2273.0   163.0    8.0    2.0   0.0   0.0  0.0   
SBIN         16.0   2179.0   2674.0   186.0    5.0    1.0   1.0   0.0  0.0   
SUNPHARMA    19.0   2333.0   2455.0   238.0   16.0    0.0   1.0   0.0  0.0   
TATAMOTORS   18.0   1853.0   2875.0   297.0   16.0    2.0   0.0   1.0  0.0   
TECHM         3.0   1405.0   1582.0   122.0   22.0    2.0   0.0   1.0  0.0   
TITAN        30.0   1990.0   2608.0   388.0   35.0   10.0   0.0   1.0  0.0   
UPL          15.0   1529.0   2068.0   162.0   10.0    4.0   1.0   0.0  0.0   
VEDL         26.0   1752.0   2784.0   456.0   38.0    3.0   2.0   0.0  0.0   
WIPRO        13.0   2267.0   2412.0   326.0   34.0    9.0   1.0   0.0  0.0   
YESBANK      15.0   1283.0   1873.0   217.0   22.0    1.0   4.0   3.0  0.0   
ZEEL         23.0   1837.0   2713.0   438.0   33.0   15.0   1.0   2.0  0.0   
Count       758.0  68120.0  73783.0  7086.0  517.0  129.0  23.0  12.0  1.0   

                      
binned         Count  
Symbol                
ADANIPORTS    2824.0  
ASIANPAINT    5062.0  
AXISBANK      5062.0  
BAJFINANCE    4994.0  
BHARTIARTL    4276.0  
BRITANNIA     5061.0  
CIPLA         5062.0  
DRREDDY       5062.0  
GRASIM        5062.0  
HCLTECH       4802.0  
HDFC          5062.0  
HDFCBANK      5062.0  
HEROMOTOCO    5062.0  
HINDALCO      5062.0  
HINDUNILVR    5062.0  
ICICIBANK     5062.0  
INFY          5062.0  
ITC           5062.0  
JSWSTEEL      3496.0  
KOTAKBANK     4741.0  
NTPC          3590.0  
ONGC          5062.0  
SBIN          5062.0  
SUNPHARMA     5062.0  
TATAMOTORS    5062.0  
TECHM         3137.0  
TITAN         5062.0  
UPL           3789.0  
VEDL          5061.0  
WIPRO         5062.0  
YESBANK       3418.0  
ZEEL          5062.0  
Count       150429.0

In [47]:
B_data.groupby('Symbol').count()

,Date,Adj_Close_Price,year,month,yr,Return,Return1,binned
Symbol,,,,,,,,
ADANIPORTS,2824,2824,2824,2824,2824,2824,2824,2824
ASIANPAINT,5062,5062,5062,5062,5062,5062,5062,5062
AXISBANK,5062,5062,5062,5062,5062,5062,5062,5062
BAJFINANCE,4994,4994,4994,4994,4994,4994,4994,4994
BHARTIARTL,4276,4276,4276,4276,4276,4276,4276,4276
BRITANNIA,5061,5061,5061,5061,5061,5061,5061,5061
CIPLA,5062,5062,5062,5062,5062,5062,5062,5062
DRREDDY,5062,5062,5062,5062,5062,5062,5062,5062
GRASIM,5062,5062,5062,5062,5062,5062,5062,5062


In [48]:
B_data.groupby('Symbol').count().binned.sum()

150429

In [ ]:
# my pivot count is correct

In [49]:
writer = pd.ExcelWriter('G:\\Best_risk_reward\\Dec2020\\Date-24-12-2020\\Data\\dayWise_oneSide_Pivot_table_Dec24_2020.xlsx',
                        engine='xlsxwriter')
B_data.to_excel(writer, sheet_name='All_stock_return_binned4')
pivot_n.to_excel(writer, sheet_name='All_Stock_Pivot_table4')  
writer.save()

In [50]:
# z_score should be by each comapny 

In [51]:
len(Company_name)

32

In [52]:
# Z_score of each comapny
# create the bins for Z-Score
dict_company = {} # 
#data_frame_list = []
new_df = pd.DataFrame()
from scipy import stats
for row in Company_name:
    B_data1 = B_data.loc[B_data['Symbol']==row]
    B_data1['Return_Zscore'] = stats.zscore(B_data1['Return1'])
    #B_data1['Return_Zscore'] =return_zscore # B_data1 contain Z_score of each comapny
    new_df=new_df.append(B_data1)
    #dict_company[row] = (return_zscore.min(), return_zscore.max())

C:\python\Anaconda3\envs\intelPython\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [53]:
new_df.head()

,Symbol,Date,Adj_Close_Price,year,month,yr,Return,Return1,binned,Return_Zscore
1,ADANIPORTS,2007-11-28,893.90,2007,11,2007,0.071659,-0.071659,4,-2.695978
2,ADANIPORTS,2007-11-29,884.20,2007,11,2007,0.010851,-0.010851,3,-0.427833
3,ADANIPORTS,2007-11-30,921.55,2007,11,2007,0.042242,0.042242,3,1.552564
4,ADANIPORTS,2007-12-03,969.30,2007,12,2007,0.051815,0.051815,4,1.909653
5,ADANIPORTS,2007-12-04,1041.45,2007,12,2007,0.074435,0.074435,4,2.753403


In [54]:
new_df['Return_Zscore'].max(),new_df['Return_Zscore'].min()

(11.879486863000453, -19.4599709161054)

In [55]:
new_df['binned'].unique()

[4, 3, 2, 6, 5, 7, 1, 8, 15]
Categories (9, int64): [1 < 2 < 3 < 4 ... 6 < 7 < 8 < 15]

In [56]:
# let's create the bins for the Z_score
B= [-35,-30,-25,-20,-15,-10,-5,0,5,10,15,20,25,30,35,40,45,50,55,60,65,70] # bins
L = [i for i in range(1,len(B))] # label

In [57]:
len(B), len(L)

(22, 21)

In [58]:
new_df['binned_Zscore']= pd.cut(x=new_df['Return_Zscore'], bins=B,labels=L)

In [59]:
new_df.head()

,Symbol,Date,Adj_Close_Price,year,month,yr,Return,Return1,binned,Return_Zscore,binned_Zscore
1,ADANIPORTS,2007-11-28,893.90,2007,11,2007,0.071659,-0.071659,4,-2.695978,7
2,ADANIPORTS,2007-11-29,884.20,2007,11,2007,0.010851,-0.010851,3,-0.427833,7
3,ADANIPORTS,2007-11-30,921.55,2007,11,2007,0.042242,0.042242,3,1.552564,8
4,ADANIPORTS,2007-12-03,969.30,2007,12,2007,0.051815,0.051815,4,1.909653,8
5,ADANIPORTS,2007-12-04,1041.45,2007,12,2007,0.074435,0.074435,4,2.753403,8


In [60]:
pivot_z= new_df.pivot_table(index=['Symbol'],values=['Return_Zscore'],columns=['binned_Zscore'],
                            aggfunc=lambda x: len(x), margins_name='Count',margins=True)

In [61]:
pivot_z.fillna(0,inplace=True)

In [62]:
pivot_z

Return_Zscore                                            \
binned_Zscore             4    5      6        7        8      9   10   
Symbol                                                                  
ADANIPORTS              0.0  0.0    1.0   1469.0   1349.0    5.0  0.0   
ASIANPAINT              0.0  0.0    2.0   2731.0   2324.0    5.0  0.0   
AXISBANK                0.0  0.0    3.0   2704.0   2345.0   10.0  0.0   
BAJFINANCE              0.0  0.0    2.0   2692.0   2286.0   14.0  0.0   
BHARTIARTL              0.0  0.0    1.0   2262.0   2008.0    4.0  1.0   
BRITANNIA               0.0  0.0    0.0   2744.0   2311.0    6.0  0.0   
CIPLA                   0.0  0.0    6.0   2672.0   2383.0    1.0  0.0   
DRREDDY                 0.0  0.0    5.0   2587.0   2467.0    3.0  0.0   
GRASIM                  0.0  0.0    5.0   2637.0   2412.0    8.0  0.0   
HCLTECH                 0.0  0.0   11.0   2395.0   2389.0    7.0  0.0   
HDFC                    0.0  0.0    2.0   2691.0   2364.0    5.0  0.0   
HDFCBANK                0.0  0.0    3.0   2687.0   2367.0    4.0  1.0   
HEROMOTOCO              0.0  0.0    1.0   2656.0   2397.0    8.0  0.0   
HINDALCO                0.0  0.0    3.0   2588.0   2468.0    3.0  0.0   
HINDUNILVR              0.0  0.0    2.0   2632.0   2424.0    4.0  0.0   
ICICIBANK               0.0  0.0    4.0   2678.0   2374.0    6.0  0.0   
INFY                    0.0  1.0    5.0   2611.0   2439.0    6.0  0.0   
ITC                     0.0  0.0    3.0   2598.0   2458.0    3.0  0.0   
JSWSTEEL                0.0  0.0    1.0   1793.0   1698.0    3.0  1.0   
KOTAKBANK               1.0  0.0    2.0   2542.0   2182.0   14.0  0.0   
NTPC                    0.0  0.0    4.0   1825.0   1757.0    4.0  0.0   
ONGC                    0.0  0.0    4.0   2605.0   2447.0    6.0  0.0   
SBIN                    0.0  0.0    3.0   2606.0   2451.0    1.0  1.0   
SUNPHARMA               0.0  0.0    3.0   2668.0   2390.0    1.0  0.0   
TATAMOTORS              0.0  0.0    2.0   2571.0   2487.0    1.0  1.0   
TECHM                   0.0  0.0    4.0   1598.0   1529.0    5.0  1.0   
TITAN                   0.0  0.0    0.0   2685.0   2366.0   11.0  0.0   
UPL                     0.0  0.0    1.0   1939.0   1841.0    8.0  0.0   
VEDL                    0.0  0.0    1.0   2696.0   2361.0    3.0  0.0   
WIPRO                   0.0  0.0    5.0   2592.0   2458.0    7.0  0.0   
YESBANK                 0.0  0.0    3.0   1789.0   1620.0    6.0  0.0   
ZEEL                    0.0  0.0   10.0   2606.0   2441.0    5.0  0.0   
Count                   1.0  1.0  102.0  78549.0  71593.0  177.0  6.0   

                         
binned_Zscore     Count  
Symbol                   
ADANIPORTS       2824.0  
ASIANPAINT       5062.0  
AXISBANK         5062.0  
BAJFINANCE       4994.0  
BHARTIARTL       4276.0  
BRITANNIA        5061.0  
CIPLA            5062.0  
DRREDDY          5062.0  
GRASIM           5062.0  
HCLTECH          4802.0  
HDFC             5062.0  
HDFCBANK         5062.0  
HEROMOTOCO       5062.0  
HINDALCO         5062.0  
HINDUNILVR       5062.0  
ICICIBANK        5062.0  
INFY             5062.0  
ITC              5062.0  
JSWSTEEL         3496.0  
KOTAKBANK        4741.0  
NTPC             3590.0  
ONGC             5062.0  
SBIN             5062.0  
SUNPHARMA        5062.0  
TATAMOTORS       5062.0  
TECHM            3137.0  
TITAN            5062.0  
UPL              3789.0  
VEDL             5061.0  
WIPRO            5062.0  
YESBANK          3418.0  
ZEEL             5062.0  
Count          150429.0

In [63]:
# sum of each company return_zscore count
#pivot_z.columns = [col[1] for col in pivot_z.columns]
#pivot_z['Total'] = pivot_z[list(pivot_z)].sum(axis=1)

In [64]:
# Z_score pivot table is also correct

In [65]:
# Number of count should be same as in pivot table
new_df.groupby('Symbol').count()

,Date,Adj_Close_Price,year,month,yr,Return,Return1,binned,Return_Zscore,binned_Zscore
Symbol,,,,,,,,,,
ADANIPORTS,2824,2824,2824,2824,2824,2824,2824,2824,2824,2824
ASIANPAINT,5062,5062,5062,5062,5062,5062,5062,5062,5062,5062
AXISBANK,5062,5062,5062,5062,5062,5062,5062,5062,5062,5062
BAJFINANCE,4994,4994,4994,4994,4994,4994,4994,4994,4994,4994
BHARTIARTL,4276,4276,4276,4276,4276,4276,4276,4276,4276,4276
BRITANNIA,5061,5061,5061,5061,5061,5061,5061,5061,5061,5061
CIPLA,5062,5062,5062,5062,5062,5062,5062,5062,5062,5062
DRREDDY,5062,5062,5062,5062,5062,5062,5062,5062,5062,5062
GRASIM,5062,5062,5062,5062,5062,5062,5062,5062,5062,5062


In [66]:
# Z_score pivot table is also correct
new_df.groupby('Symbol').count().binned_Zscore.sum()

150429

In [67]:
pivot_z.shape

(33, 8)

In [68]:
# lets save in the excel format
writer1 = pd.ExcelWriter('G:\\Best_risk_reward\\Dec2020\\Date-24-12-2020\\Data\\pivot_n_Z_Dec24_2020.xlsx', engine='xlsxwriter')
pivot_n.to_excel(writer1, sheet_name='pivot_for_return4')
pivot_z.to_excel(writer1, sheet_name='pivot_for_Zscore4')
new_df.to_excel(writer1, sheet_name='Zscore_with_binned4')
writer1.save()

In [69]:
#now create the pivot table for each of the comapny
Compnay_name=[]
r_large_pivot = []
z_large_pivot =[]
for row in Company_name:
    data_as1 = new_df[new_df['Symbol']==row]
    #print('='*40+' nlarge of Each company ' + str(row)+'='*40)
    
    B_data2=data_as1.nlargest(15, "Return")
    
    pivot_Z_large = B_data2.pivot_table(index=['Symbol'],values=['Return_Zscore'],columns=['binned_Zscore'],
                                        aggfunc=lambda x: len(x),margins_name='Count',margins=True)
    
    #pivot_Z_large.columns = [col[1] for col in pivot_Z_large.columns]
    #pivot_Z_large['Total'] = pivot_Z_large[list(pivot_Z_large)].sum(axis=1)
    
    print('pivot table for Z_score n-large :\n',pivot_Z_large)
    print('-'*100, '\n')
    
    pivot_n_large= B_data2.pivot_table(index=['Symbol'],values=['Return'],columns=['binned'],aggfunc=lambda x: len(x),
                                      margins_name='Count',margins=True)
    
    
    #pivot_n_large.columns = [col[1] for col in pivot_n_large.columns]
   # pivot_n_large['Total_n'] = pivot_n_large[list(pivot_n_large)].sum(axis=1)
    print('pivot table for n-large abs Return :\n',pivot_n_large)
    print('='*100, '\n')
    Compnay_name.append(row)
    z_large_pivot.append(pivot_Z_large)  
    r_large_pivot.append(pivot_n_large)


pivot table for Z_score n-large :
               Return_Zscore                     
binned_Zscore             6    7    8    9 Count
Symbol                                          
ADANIPORTS              1.0  5.0  4.0  5.0  15.0
Count                   1.0  5.0  4.0  5.0  15.0
---------------------------------------------------------------------------------------------------- 

pivot table for n-large abs Return :
            Return                
binned          5    6    7 Count
Symbol                           
ADANIPORTS   10.0  3.0  2.0  15.0
Count        10.0  3.0  2.0  15.0

pivot table for Z_score n-large :
               Return_Zscore                     
binned_Zscore             6    7    8    9 Count
Symbol                                          
ASIANPAINT              2.0  2.0  6.0  5.0  15.0
Count                   2.0  2.0  6.0  5.0  15.0
---------------------------------------------------------------------------------------------------- 

pivot table for n-large a

pivot table for n-large abs Return :
            Return                
binned          4    5    6 Count
Symbol                           
HEROMOTOCO    7.0  4.0  4.0  15.0
Count         7.0  4.0  4.0  15.0

pivot table for Z_score n-large :
               Return_Zscore                     
binned_Zscore             6    7    8    9 Count
Symbol                                          
HINDALCO                3.0  3.0  6.0  3.0  15.0
Count                   3.0  3.0  6.0  3.0  15.0
---------------------------------------------------------------------------------------------------- 

pivot table for n-large abs Return :
          Return                
binned        5    6    7 Count
Symbol                         
HINDALCO   12.0  2.0  1.0  15.0
Count      12.0  2.0  1.0  15.0

pivot table for Z_score n-large :
               Return_Zscore                
binned_Zscore             6    8    9 Count
Symbol                                     
HINDUNILVR              2.0  9.0  4.0  15.

            Return                
binned          5    6    8 Count
Symbol                           
TATAMOTORS   12.0  2.0  1.0  15.0
Count        12.0  2.0  1.0  15.0

pivot table for Z_score n-large :
               Return_Zscore                          
binned_Zscore             6    7    8    9   10 Count
Symbol                                               
TECHM                   4.0  1.0  4.0  5.0  1.0  15.0
Count                   4.0  1.0  4.0  5.0  1.0  15.0
---------------------------------------------------------------------------------------------------- 

pivot table for n-large abs Return :
        Return                
binned      5    6    8 Count
Symbol                       
TECHM    12.0  2.0  1.0  15.0
Count    12.0  2.0  1.0  15.0

pivot table for Z_score n-large :
               Return_Zscore                 
binned_Zscore             7    8     9 Count
Symbol                                      
TITAN                   1.0  3.0  11.0  15.0
Count           

In [70]:
writer3 = pd.ExcelWriter('G:\\Best_risk_reward\\Dec2020\\Date-24-12-2020\\Data\\n_large_of each_comapy_Dec24_2020.xlsx', engine='xlsxwriter')
writer4 = pd.ExcelWriter('G:\\Best_risk_reward\\Dec2020\\Date-24-12-2020\\Data\\n_large_zscore_each_company_Dec24_2020.xlsx', engine='xlsxwriter')

for name,df1,df2 in zip(Compnay_name,r_large_pivot,z_large_pivot):
    
    df1.to_excel(writer3, sheet_name='pivot of n large '+ str(name))
    df2.to_excel(writer4, sheet_name='pivotof_n_Zscore '+ str(name))

writer3.save()
writer4.close() 